# TPS 12 - Feature Engineering

In [1]:
# Global variables for testing changes to this notebook quickly
RANDOM_SEED = 0
NUM_FOLDS = 3
TRAIN_SIZE = 500000

In [2]:
import numpy as np
import pandas as pd
import time
import os
import pyarrow
import gc

# Model/Evaluation
from sklearn.base import clone
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, recall_score
from sklearn.inspection import permutation_importance
from lightgbm import LGBMClassifier

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

# Load and Prepare Data

In [3]:
# Load full training data
train = pd.read_feather('../data/train.feather')

# Drop low/no variance 
train.drop(["Soil_Type7", "Id", "Soil_Type15"], axis=1, inplace=True)
train = train[train.Cover_Type != 5]

# Label Encoding
new_encoder = LabelEncoder()
train["Cover_Type"] = new_encoder.fit_transform(train["Cover_Type"])

# Split synthetic data
train, test = train_test_split(
    train, 
    train_size = TRAIN_SIZE, 
    random_state = RANDOM_SEED,
    stratify = train['Cover_Type'],
)
y_train = train['Cover_Type']


# features, data structure for summary scores
features = [x for x in train.columns if x not in ['Id','Cover_Type']]
nonsoil = [x for x in features if not x.startswith('Soil_Type')]
new_rows = list()
gc.collect()

print(f'Training Size: {train.shape[0]} rows, {train.shape[1]} cols')
print(f'Holdout Size: {test.shape[0]} rows, {test.shape[1]} cols\n')

Training Size: 500000 rows, 53 cols
Holdout Size: 3499999 rows, 53 cols



# Model 

In [4]:
# LightGBM Classifier
lgbm_pipeline = make_pipeline(
    LGBMClassifier(
        unbalanced_sets = True,
        metric = 'multi_logloss',
        random_state = RANDOM_SEED,
    ),
)

# Scoring Function

In [5]:
def score_features(sklearn_model, processing = None):
    
    # Original Training/Test Split
    features = [x for x in train.columns if x not in ['Id','Cover_Type']]
    X_temp, X_test = train[features], test[features]
    y_temp, y_test = train['Cover_Type'], test['Cover_Type']
    
    # Feature Engineering
    if processing:
        X_temp = processing(X_temp)
        X_test = processing(X_test)
    features = [x for x in X_temp.columns]
    
    # Store the out-of-fold predictions
    test_preds = np.zeros((X_test.shape[0],6))
    oof_preds = np.zeros((X_temp.shape[0],))
    fi_scores = np.zeros((X_temp.shape[1],))
    scores, times = np.zeros(NUM_FOLDS), np.zeros(NUM_FOLDS)
    
    # Stratified k-fold cross-validation
    skf = StratifiedKFold(n_splits = NUM_FOLDS, shuffle = True, random_state = RANDOM_SEED)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X_temp,y_temp)):
       
        # Training and Validation Sets
        X_train, X_valid = X_temp.iloc[train_idx], X_temp.iloc[valid_idx]
        y_train, y_valid = y_temp.iloc[train_idx], y_temp.iloc[valid_idx]
        
        # Create model
        start = time.time()
        model = clone(sklearn_model)
        model.fit(X_train, y_train)

        # Permutation Importance
        result = permutation_importance(
            model, X_valid, y_valid, 
            n_repeats=10, random_state=RANDOM_SEED
        )
        fi_scores += result.importances_mean / NUM_FOLDS

        # validation/holdout predictions
        valid_preds = np.ravel(model.predict(X_valid))
        oof_preds[valid_idx] = valid_preds
        test_preds += model.predict_proba(X_test)

        # Save scores and times
        scores[fold] = accuracy_score(y_valid, valid_preds)
        end = time.time()
        times[fold] = end-start
        print(f'Fold {fold} Accuracy:  {round(scores[fold], 5)} in {round(end-start,2)}s.')
        time.sleep(0.5)
    
    nonsoil = [x for x in X_test.columns if not x.startswith('Soil_Type')]
    test_preds = np.argmax(test_preds, axis = 1)
    test_score = accuracy_score(y_test, test_preds)
    #print('\n'+model.__class__.__name__)
    print("Train Accuracy:", round(scores.mean(), 5))
    print('Test Accuracy:', round(test_score, 5))
    print(f'Training Time: {round(times.sum(), 2)}s')
    
    fi_scores = pd.Series(
        data = fi_scores, 
        index = features
    ).loc[nonsoil].sort_values()
    
    return scores.mean(), oof_preds, test_score, fi_scores

# Baseline

In [6]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    lgbm_pipeline
)

new_rows.append((
    'Baseline', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.94267 in 419.04s.
Fold 1 Accuracy:  0.93561 in 356.14s.
Fold 2 Accuracy:  0.94458 in 417.37s.
Train Accuracy: 0.94095
Test Accuracy: 0.95099
Training Time: 1192.55s


Hillshade_9am                        -0.000010
Aspect                                0.000005
Hillshade_3pm                         0.000013
Wilderness_Area2                      0.000031
Slope                                 0.000074
Hillshade_Noon                        0.000206
Wilderness_Area4                      0.002571
Horizontal_Distance_To_Hydrology      0.008780
Wilderness_Area1                      0.013618
Vertical_Distance_To_Hydrology        0.015172
Wilderness_Area3                      0.016961
Horizontal_Distance_To_Fire_Points    0.026114
Horizontal_Distance_To_Roadways       0.040254
Elevation                             0.451731
dtype: float64

# Feature Engineering

## 1 . Fix Aspect and Hillshade Values

In [7]:
# Fix aspect
def fix_ranges(data):
    
    df = data.copy()
    
    # Fix Aspect
    df["Aspect"][df["Aspect"] < 0] += 360
    df["Aspect"][df["Aspect"] > 359] -= 360
    
    # Fix Hillshade
    df.loc[df["Hillshade_9am"] < 0, "Hillshade_9am"] = 0
    df.loc[df["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0
    df.loc[df["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0
    df.loc[df["Hillshade_9am"] > 255, "Hillshade_9am"] = 255
    df.loc[df["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255
    df.loc[df["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255
    
    return df

In [8]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    lgbm_pipeline, 
    fix_ranges
)

new_rows.append((
    'Fix_Range', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.94309 in 424.61s.
Fold 1 Accuracy:  0.94833 in 433.42s.
Fold 2 Accuracy:  0.94699 in 423.63s.
Train Accuracy: 0.94614
Test Accuracy: 0.95498
Training Time: 1281.67s


Slope                                -0.000003
Hillshade_9am                         0.000017
Aspect                                0.000023
Wilderness_Area2                      0.000050
Hillshade_3pm                         0.000125
Hillshade_Noon                        0.000330
Wilderness_Area4                      0.004230
Horizontal_Distance_To_Hydrology      0.009734
Vertical_Distance_To_Hydrology        0.016932
Wilderness_Area1                      0.020593
Wilderness_Area3                      0.026580
Horizontal_Distance_To_Fire_Points    0.028732
Horizontal_Distance_To_Roadways       0.043813
Elevation                             0.455661
dtype: float64

## 2. Water Distance Features

In [9]:
# Distance to Water
def water_distance_features(data):
    df = data.copy()
    
    # use float64 for squaring
    df["Horizontal_Distance_To_Hydrology"] = df["Horizontal_Distance_To_Hydrology"].astype('float64')
    df["Vertical_Distance_To_Hydrology"] = df["Vertical_Distance_To_Hydrology"].astype('float64')
    
    # compute metrics
    df["Hydro_Taxicab"] = np.abs(df["Horizontal_Distance_To_Hydrology"]) + np.abs(df["Vertical_Distance_To_Hydrology"])
    df["Hydro_Euclid"] = (df["Horizontal_Distance_To_Hydrology"]**2 + np.abs(df["Vertical_Distance_To_Hydrology"])**2)**0.5
    
    # convert back
    df["Horizontal_Distance_To_Hydrology"] = df["Horizontal_Distance_To_Hydrology"].astype('float32')
    df["Vertical_Distance_To_Hydrology"] = df["Vertical_Distance_To_Hydrology"].astype('float32')
    df["Hydro_Taxicab"] = df["Hydro_Taxicab"].astype('float32')
    df["Hydro_Euclid"] = df["Hydro_Euclid"].astype('float32')
    
    return df

In [10]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    lgbm_pipeline, 
    water_distance_features
)

new_rows.append((
    'Water_Dist', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.94617 in 447.55s.
Fold 1 Accuracy:  0.94649 in 462.88s.
Fold 2 Accuracy:  0.94667 in 446.98s.
Train Accuracy: 0.94644
Test Accuracy: 0.95385
Training Time: 1357.41s


Wilderness_Area2                      0.000041
Hillshade_9am                         0.000059
Slope                                 0.000076
Hillshade_3pm                         0.000144
Aspect                                0.000145
Hillshade_Noon                        0.000590
Horizontal_Distance_To_Hydrology      0.001769
Wilderness_Area4                      0.003656
Vertical_Distance_To_Hydrology        0.013490
Wilderness_Area1                      0.018238
Hydro_Euclid                          0.020134
Wilderness_Area3                      0.022744
Hydro_Taxicab                         0.025357
Horizontal_Distance_To_Fire_Points    0.027925
Horizontal_Distance_To_Roadways       0.044058
Elevation                             0.455153
dtype: float64

## 3. Count Features

In [11]:
# Create count features
def count_features(data):
    
    df = data.copy()
    soil_features = [x for x in df.columns if x.startswith("Soil_Type")]
    wilderness_features = [x for x in df.columns if x.startswith("Wilderness_Area")]

    # Count features
    df["Soil_Count"] = df[soil_features].apply(sum, axis=1)
    df["Wilderness_Count"] = df[wilderness_features].apply(sum, axis=1)
    
    return df

In [12]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    lgbm_pipeline, 
    count_features
)

new_rows.append((
    'Count', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.92137 in 356.05s.
Fold 1 Accuracy:  0.9473 in 372.48s.
Fold 2 Accuracy:  0.92728 in 378.96s.
Train Accuracy: 0.93198
Test Accuracy: 0.94694
Training Time: 1107.49s


Slope                                -0.000021
Wilderness_Area2                      0.000073
Hillshade_3pm                         0.000077
Aspect                                0.000084
Hillshade_9am                         0.000180
Hillshade_Noon                        0.000186
Wilderness_Count                      0.000760
Wilderness_Area4                      0.002198
Horizontal_Distance_To_Hydrology      0.007184
Vertical_Distance_To_Hydrology        0.013059
Wilderness_Area1                      0.013746
Wilderness_Area3                      0.016464
Horizontal_Distance_To_Fire_Points    0.022180
Soil_Count                            0.030852
Horizontal_Distance_To_Roadways       0.039666
Elevation                             0.443748
dtype: float64

## 4. Shade Features

In [13]:
# Create Shade features
def new_shade_features(data):
    
    df = data.copy()
    shade_features = ['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']

    # Hillshade
    df["Hillshade_Avg"] = df[shade_features].mean(axis=1)
    df['Hillshade_Range'] = df[shade_features].max(axis=1) - df[shade_features].min(axis=1)
    
    return df

In [14]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    lgbm_pipeline, 
    new_shade_features
)

new_rows.append((
    'Shade_Features', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95032 in 458.16s.
Fold 1 Accuracy:  0.94062 in 386.68s.
Fold 2 Accuracy:  0.94633 in 415.66s.
Train Accuracy: 0.94576
Test Accuracy: 0.95348
Training Time: 1260.51s


Aspect                               -0.000116
Slope                                -0.000055
Wilderness_Area2                      0.000043
Hillshade_Noon                        0.000374
Hillshade_9am                         0.000445
Hillshade_Avg                         0.000490
Hillshade_Range                       0.000966
Hillshade_3pm                         0.001041
Wilderness_Area4                      0.002109
Horizontal_Distance_To_Hydrology      0.009246
Wilderness_Area1                      0.014744
Vertical_Distance_To_Hydrology        0.016431
Wilderness_Area3                      0.018731
Horizontal_Distance_To_Fire_Points    0.027436
Horizontal_Distance_To_Roadways       0.042960
Elevation                             0.454820
dtype: float64

## 5. Distance Interactions

In [15]:
def distance_interactions(data):
    
    df = data.copy()
    df['Hydro_Fire_1'] = df['Horizontal_Distance_To_Hydrology'] + df['Horizontal_Distance_To_Fire_Points']
    df['Hydro_Fire_2'] = abs(df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Fire_Points'])
    df['Hydro_Road_1'] = abs(df['Horizontal_Distance_To_Hydrology'] + df['Horizontal_Distance_To_Roadways'])
    df['Hydro_Road_2'] = abs(df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Roadways'])
    df['Fire_Road_1'] = abs(df['Horizontal_Distance_To_Fire_Points'] + df['Horizontal_Distance_To_Roadways'])
    df['Fire_Road_2'] = abs(df['Horizontal_Distance_To_Fire_Points'] - df['Horizontal_Distance_To_Roadways'])
    return df

In [16]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    lgbm_pipeline, 
    distance_interactions
)

new_rows.append((
    'Dist_Interactions', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.94798 in 474.62s.
Fold 1 Accuracy:  0.94945 in 488.0s.
Fold 2 Accuracy:  0.94924 in 461.89s.
Train Accuracy: 0.94889
Test Accuracy: 0.9561
Training Time: 1424.5s


Hillshade_3pm                        -0.000006
Slope                                 0.000010
Hillshade_9am                         0.000011
Aspect                                0.000011
Wilderness_Area2                      0.000039
Hillshade_Noon                        0.000356
Hydro_Road_1                          0.003022
Wilderness_Area4                      0.004762
Fire_Road_2                           0.005065
Horizontal_Distance_To_Hydrology      0.005758
Fire_Road_1                           0.008513
Hydro_Fire_1                          0.008773
Horizontal_Distance_To_Fire_Points    0.008861
Hydro_Fire_2                          0.010592
Hydro_Road_2                          0.010843
Vertical_Distance_To_Hydrology        0.017000
Wilderness_Area1                      0.017030
Wilderness_Area3                      0.022000
Horizontal_Distance_To_Roadways       0.024819
Elevation                             0.456269
dtype: float64

## 6. Elevation Interactions

In [17]:
def elevation_interactions(data):
    
    df = data.copy()
    df['EHiElv'] = df['Horizontal_Distance_To_Roadways'] * df['Elevation']
    df['EViElv'] = df['Vertical_Distance_To_Hydrology'] * df['Elevation']
    df['EVDtH'] = df.Elevation - df.Vertical_Distance_To_Hydrology
    df['EHDtH'] = df.Elevation - df.Horizontal_Distance_To_Hydrology * 0.2
    
    # Throwaway Features most likely
    df['Highwater'] = (df.Vertical_Distance_To_Hydrology < 0).astype(int)
    df['Hillshade_3pm_is_zero'] = (df.Hillshade_3pm == 0).astype(int)
    
    return df

In [18]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    lgbm_pipeline, 
    elevation_interactions
)

new_rows.append((
    'Elev_Interactions', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.94661 in 476.06s.
Fold 1 Accuracy:  0.91801 in 420.54s.
Fold 2 Accuracy:  0.94376 in 458.78s.
Train Accuracy: 0.93612
Test Accuracy: 0.95015
Training Time: 1355.37s


Hillshade_9am                        -9.280016e-05
EHiElv                               -4.240025e-05
Hillshade_3pm                        -1.320028e-05
Hillshade_3pm_is_zero                 1.999996e-07
Aspect                                2.519981e-05
Highwater                             2.599995e-05
Slope                                 5.819983e-05
Wilderness_Area2                      6.620004e-05
EViElv                                9.980032e-05
Hillshade_Noon                        2.287996e-04
Wilderness_Area4                      4.207399e-03
Horizontal_Distance_To_Hydrology      5.397001e-03
Vertical_Distance_To_Hydrology        8.866384e-03
Wilderness_Area1                      1.367940e-02
Wilderness_Area3                      1.824220e-02
Horizontal_Distance_To_Fire_Points    2.462860e-02
Horizontal_Distance_To_Roadways       4.048001e-02
EHDtH                                 1.282565e-01
EVDtH                                 1.741387e-01
Elevation                      

# Summary

In [19]:
pd.DataFrame.from_records(
    data = new_rows,
    columns = ['features','cv_scores','holdout','recall_0', 'recall_1','recall_2','recall_3','recall_4','recall_5']
).sort_values('holdout')

,features,cv_scores,holdout,recall_0,recall_1,recall_2,recall_3,recall_4,recall_5
3,Count,0.931984,0.946935,0.941629,0.946241,0.816220,0.0,0.425070,0.649107
6,Elev_Interactions,0.936124,0.950146,0.943319,0.954920,0.805060,0.0,0.413866,0.597071
0,Baseline,0.940954,0.950986,0.951024,0.958163,0.816996,0.0,0.387955,0.575100
4,Shade_Features,0.945758,0.953476,0.954162,0.961961,0.835146,0.0,0.420168,0.608763
2,Water_Dist,0.946442,0.953845,0.955181,0.961688,0.838620,0.0,0.421569,0.627393
1,Fix_Range,0.946136,0.954976,0.954871,0.961363,0.841727,0.0,0.405462,0.620069
5,Dist_Interactions,0.948890,0.956099,0.958037,0.963814,0.837598,0.0,0.434874,0.640884
